In [3]:
import os
import openai
import time
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
from ute import init_openai, read_page_list, rdweb, embedpagelist, sepstr
from ute import SEP, get_embedding, cosine_similarity
import numpy as np

# Define the md() function to display markdown text
from IPython.display import display, Markdown
def md(s):
    display(Markdown(s))


(Client, LLM)=init_openai()
webfolder="general"
datafolder="data"+"/"+webfolder+"/"
DataFile=datafolder+"data"+".txt"
TextListFile=datafolder+webfolder+"_textlist.txt"
UploadListFile=datafolder+webfolder+"_uploadlist.txt"
UploadIDFile=datafolder+webfolder+"_uploadid.txt"
EmbeddingFile=datafolder+webfolder+"_embedding.npz"

In [4]:
s="|Contents|File Name|\n|--|--|\n\
|Web folder name|"+webfolder+"|\n\
|Data folder name|"+datafolder+"|\n\
|Data file name|"+DataFile+"|\n\
|Text list file name|"+TextListFile+"|\n\
|Upload list file name|"+UploadListFile+"|\n\
|Upload ID file name|"+UploadIDFile+"|\n\
|Embedding file name|"+EmbeddingFile+"|\n\
|Client|"+str(Client)+"|\n\
|LLM|"+str(LLM)+"|\n\
\n\n"
md(s)
s="|Variable name|Description|\n|--|--|\n\
|webfolder|Web folder name|\n\
|datafolder|Data folder name|\n\
|DataFile|Data file name|\n\
|TextListFile|Text list file name|\n\
|UploadListFile|Upload list file name|\n\
|UploadIDFile|Upload ID file name|\n\
|EmbeddingFile|Embedding file name|\n\
|Client|OpenAI client|\n\
|LLM|Language model|\n\
UploadList|List of file names to upload|\n\
UploadID|List of OpenAI file IDs (created when files are uploaded)|\n\
TextList|List of text files (as pairs of two lines, URL and title)|\n\
Embedding|Embedding matrix|\n\
\n\n"
md(s)


|Contents|File Name|
|--|--|
|Web folder name|general|
|Data folder name|data/general/|
|Data file name|data/general/data.txt|
|Text list file name|data/general/general_textlist.txt|
|Upload list file name|data/general/general_uploadlist.txt|
|Upload ID file name|data/general/general_uploadid.txt|
|Embedding file name|data/general/general_embedding.npz|
|Client|<openai.OpenAI object at 0x120616a50>|
|LLM|text-embedding-ada-002|




|Variable name|Description|
|--|--|
|webfolder|Web folder name|
|datafolder|Data folder name|
|DataFile|Data file name|
|TextListFile|Text list file name|
|UploadListFile|Upload list file name|
|UploadIDFile|Upload ID file name|
|EmbeddingFile|Embedding file name|
|Client|OpenAI client|
|LLM|Language model|
UploadList|List of file names to upload|
UploadID|List of OpenAI file IDs (created when files are uploaded)|
TextList|List of text files (as pairs of two lines, URL and title)|
Embedding|Embedding matrix|




## Save the text files locally and upload to OpenAI ##
We upload the files to the file repository of the Client. They are not yet associated with any Thread.  A new thread is created after the Query is set.

In [5]:
# The following function reads the names of the URLs already uploaded to OpenAI
def read_upload_list(UploadListFile):
    UploadList=[]
    if os.path.exists(UploadListFile):
        with open(UploadListFile, 'r') as f:
            UploadList = f.read().splitlines()
    return UploadList

UploadList=read_upload_list(UploadListFile)

# The following function reads the IDs of the URLs already uploaded to OpenAI
def read_upload_id(UploadIDFile):
    UploadID=[]
    if os.path.exists(UploadIDFile):
        with open(UploadIDFile, 'r') as f:
            UploadID = f.read().splitlines()
    return UploadID

UploadID=read_upload_id(UploadIDFile)
#
# The following function extracts a file name from the URL.  The file name is
# used to name the text file that will be uploaded to OpenAI
# The file name is also used to check if the file has already been uploaded
# Use the last part of the URL as the file name
def get_file_name(url):
    url_parts=url.split("/")
    filename=url_parts[-1]
    return filename

# Read the URL addresses to be read into text files from the file TextListFile
# Ignore the URLs that have already been uploaded to OpenAI

TextList=read_page_list(TextListFile)
NewUploads=[]
NewUploadIDs=[]
for (itext,pair) in enumerate(TextList):
    (url,title)=pair
    txtfile=get_file_name(url)+".txt"
    if txtfile in UploadList:
        print("Skipping %s because it has already been uploaded"%txtfile)
        continue
    # Rad the text from the URL address and save into the text file
    rdweb(url, datafolder+txtfile)
    print("Uploading %s"%txtfile, end="")
    File = Client.files.create(
        file=open(datafolder+txtfile, "rb"),
        purpose='assistants'
)    
    print(" --- Uploaded")
    time.sleep(1)
    NewUploads.append(txtfile)
    NewUploadIDs.append(File.id)
with open(UploadListFile, 'a') as f:
    for item in NewUploads:
        f.write("%s\n" % item)
if len(NewUploads)>0:
    print("Updated %s (Names of the Uploaded Files)"%UploadListFile)
else:
    print("No new files uploaded")
# Append the new IDs to the file UploadIDFile
with open(UploadIDFile, 'a') as f:
    for item in NewUploadIDs:
        f.write("%s\n" % item)
if len(NewUploadIDs)>0:
    print("Updated %s (OpenAI IDs of the uploaded files)"%UploadIDFile)
else:
    print("No new IDs uploaded")
#
UploadID=UploadID+NewUploadIDs
UploadList=UploadList+NewUploads

Skipping the-requiem-for-a-dream-israels-untaken.txt because it has already been uploaded
Skipping the-importance-of-elite-consensus.txt because it has already been uploaded
Skipping openai-astounds-us-again.txt because it has already been uploaded
Skipping why-did-elon-musk-buy-twitter.txt because it has already been uploaded
Skipping elon-musk-the-spaceman.txt because it has already been uploaded
Skipping let-us-talk-about-elon-musk.txt because it has already been uploaded
Skipping despicable-acts-part-2.txt because it has already been uploaded
Skipping despicable-deeds.txt because it has already been uploaded
Skipping rogue-age-and-climate-change-unpredictable.txt because it has already been uploaded
Skipping rogue-age-accessory-1-population.txt because it has already been uploaded
Skipping rogue-renaissance-on-globe-with-upheavals.txt because it has already been uploaded
Skipping the-great-stagnation-ends-but-for.txt because it has already been uploaded
Skipping when-the-rivers-run

## Read Assistant ID from file or create new ##

In [6]:
# Check if the DataFile exists
if os.path.isfile(DataFile):
    # Read the Assistant ID from the file
    f = open(DataFile, "r")
    AssistantID = f.read()
    f.close()
    print("Retrieve Assistant ID: %s"%AssistantID)
else:
    # Create a new Assistant
    Assistant = Client.beta.assistants.create(
    name="Assistant for "+webfolder,
    model="gpt-4-1106-preview",
    instructions="You are a school teacher answering students questions about the course material provided to you in text files. If the \
 response is not in the text files, you can respond with 'I don't know'.",
    tools=[{"type": "retrieval"}],
    # file_ids=UploadID
    )
    AssistantID = Assistant.id
    # Write the Assistant ID to the file
    f = open(DataFile, "w")
    f.write(AssistantID)
    f.close()
    print("Created Assistant ID: %s"%AssistantID)

Retrieve Assistant ID: asst_LCdnKNcUtkxVg1aDoiQ9vHCX


## Generate Embeddings ##
At the time I am writing this, I have 33 blog posts under the 'general' tab on my Substack page.  Each blog post is in a separate text file.  This makes it 33 text files.  The OpenAI does not allow to attach more than 20 files.  Since I cannot attach all 33, I have to find the files that are most relevant to the question and attach only those files.  These files can be selected in a number of ways. Before we consider different options, we need to generate embeddings.  In the following cell, I do the following:
* Load embeddings from the `npz` file if it exists
* If there is no `npz` file, embed all files and store with metadata
* Check if there are new text files in the TextList
* Embed the new text files and append them to the end of the embeddings

In [7]:
def embedandsave():
    embeddings=embedpagelist(TextList)
    metadata={"model":LLM, "webfolder":webfolder, "textlist":TextListFile, "texts":TextList}
    np.savez(EmbeddingFile,data=embeddings, metadata=metadata)
    return embeddings, metadata

if not os.path.exists(EmbeddingFile):
    (embeddings, metadata)=embedandsave()
# Load embeddings from file
with np.load(EmbeddingFile, allow_pickle=True) as data:
    embeddings=data['data']
    metadata=data['metadata'].item()

if not metadata["model"]==LLM:
    (embeddings, metadata)=embedandsave()

# Generate the liost of embedded textx that are in the npz file:
embedded_texts=[]
for pair in metadata["texts"]:
    embedded_texts.append(pair[1])

# Now check if there are any new texts to be embedded
update=False
for (itext,pair) in enumerate(TextList):
    url=pair[0]
    texttitle=pair[1]
    # print("Check '"+texttitle+"' --> ", end="")
    if texttitle not in embedded_texts:
        print("Update "+texttitle)
        text=rdweb(url, None)
        sa=sepstr(text, SEP)
        for (isegment, s) in enumerate(sa):
            v=get_embedding(s)
            w=np.append(v,[itext,isegment])
            embeddings=np.vstack((embeddings,w))
        metadata["texts"].append(pair)
        update=True
    # else:
        # print("OK")
if update:
    # Save the embeddings to a file
    np.savez(EmbeddingFile,data=embeddings, metadata=metadata)



## Special Questions ##

In [20]:
OpMode=0
OpTask=["check", "answer", "summarise"][OpMode]

def checkmode():
    global OpMode
    OpMode=0
    print("Switched to check mode")
    return
def answermode():
    global OpMode
    OpMode=1
    print("Switched to answer mode")
    return
def summarisemode():
    global OpMode
    OpMode=2
    print("Switched to summarise mode")
    return

Specials=["Only provide the URL", "Prepare to answer questions", "Summarise the pages"]
SpecialFunctions=[checkmode, answermode, summarisemode]
SpecialEmbeds=[get_embedding(s) for s in Specials]


## Prompt ##
The following cell is the prompt I use.  It will be saved to the top of the log file.

In [9]:
q="Query goes here"
Prompt="You are school teacher answering the questions from your pupils.  \
  Please answer the following question from one student:\n\n'%s'\n\n\
Use the attached text files to answer the question. \
      The information may not be directly available and you may have to interpret the \
        information to answer the query.  Try to give an answer but make sure , \
        you respond with 'I don't know' when there is no answer."
md(Prompt%q+"\n\n")

You are school teacher answering the questions from your pupils.    Please answer the following question from one student:

'Query goes here'

Use the attached text files to answer the question.       The information may not be directly available and you may have to interpret the         information to answer the query.  Try to give an answer but make sure ,         you respond with 'I don't know' when there is no answer.



## Logging our session ##
I will store the query, the associated files, and the OpenAI response in a file.  This file will be in markdown format.

In [10]:
Logfile=datafolder+webfolder+"_log.md"
Log=open(Logfile, "a")
Log.write("**Prompt**\n\n"+Prompt+"\n\n")
Log.close()

## The query string ##
In this notebook, our question is defined in the following cell.  For a new question, enter a new line with the new question content.  This process can be interactive with buttons and text entry windows as I did it in `embeddings.ipynb`.  Personally, I find it easier to do it manually by changiung the string in the cell.

In [11]:
Query = "Why did Duke Energy convert a gas turbine to hydrogen?" 
Query="What time of the day was the Turkish Earthquake?"
Query="Who is Oliver Anthony?"
Query="Does eVinci microreactor need cooling water?"
Query="Was there an earthquake in Japan in 2011?"
Query="Is it expensive to build earthquake resistant buildings?"
Query="Was there an earthquake in Turkey in 2023?"
Query="Why did so many people die in the Turkish earthquake?"
Query="Is there going to be a war between the US and China?"
Query="How much Local Government Debt is there in China?"
Query="DOE Hydrogen Program"
query_embedding=get_embedding(Query)
#
Log=open(Logfile, "a")


## Operation Mode ##
This can do one of the following things deopendng on the value of the `OpMode`.  

|Mode|Action|
|:--:|----|
|0 - Check|Do not answer. Only return the URLs of the pages that match |
|1 - Answer|Find the best-match URLs, ask OpenAI to reply using thuse URL files|
|2 - Summarise|Find the best-match URLs, ask OpenAI to summarise them| 

The default mode is `check`.  One can change the mode by entering one of the following strings (or similar) as the query:

In [12]:
table="|Query text|For Mode|\n|--|--|\n"
for s in Specials:
    table+="|%s|%s|\n"%(s, ["Check", "Answer", "Summarise"][Specials.index(s)])
md(table)
md("\n\n")

|Query text|For Mode|
|--|--|
|Only provide the URL|Check|
|Prepare to answer questions|Answer|
|Summarise the pages|Summarise|


## Check the query, change `OpMode` is necessary ##

In [13]:

# Check if the Query is similar to one of the strings in Specials:
for (i,s) in enumerate(Specials):
    if cosine_similarity(query_embedding, SpecialEmbeds[i])>0.9:
        SpecialFunctions[i]()
        break



## Select which files to attach ##
The easiest thing to do is to attach the top N files that are the most relevant to the query.  I will start with N=10.  The top 10 files are determined by computing the cosine-similarity of their segments to the query.

The file score will be equal to the score of the closest segment in that file.

In [14]:
N=3
textsimilarity=np.ones(len(embeddings))*(-1)
for i in range(len(embeddings)):
    similarity = cosine_similarity(embeddings[i][:-2], query_embedding)
    itext=int(embeddings[i][-2])
    if similarity>textsimilarity[itext]:
        textsimilarity[itext]=similarity
topN=(np.argsort(textsimilarity)[::-1])[0:N]
topN=topN.astype(int)
topN=topN.tolist()


In [15]:
s="**Query** : "+Query
s=s+"\n\nThe following are the top %d most relevant texts:\n\n"%N
s=s+"|itext|similarity|title|\n|--|--|--|\n"
for i in topN:
    s=s+"|%d|%f|%s|\n"%(i,textsimilarity[i],metadata["texts"][i][1])
s=s+"\n\nWe will attach these files to the thread.\n\n"
md(s)
#Log.write(s)

**Query** : DOE Hydrogen Program

The following are the top 3 most relevant texts:

|itext|similarity|title|
|--|--|--|
|6|0.821517|Despicable Acts - Part 2|
|1|0.800706|The importance of elite consensus|
|9|0.780822|ROGUE Age Accessory #1 - Population|


We will attach these files to the thread.



## Create Thread ##
Create a thread with the query and the top N relevant files as information.

In [16]:

def thread():
    nextquestion=Query
    content="Please answer the following question:\n\n'%s'\n\n\
    Use the attached text files to answer the question. \
      The information may not be directly available and you may have to interpret the \
        information to answer the query.  Try to give an answer but make sure , \
        you respond with 'I don't know' when there is no answer."%nextquestion
    Thread = Client.beta.threads.create(
      messages=[
        {
          "role": "user",
          "content": content,
          "file_ids": [UploadID[i] for i in topN]
        }
    ]
    )
    return Thread


if OpTask=="answer":
    Thread=thread()

## Run the Query ##

In [17]:
def runquery():
    run = Client.beta.threads.runs.create(
            thread_id=Thread.id,
            assistant_id=AssistantID,
            instructions=""
            )

    print("RUN STARTED.  run id: ", run.id)

    print(run.status, end=" ")
    while run.status != "completed":
        run = Client.beta.threads.runs.retrieve(thread_id=run.thread_id, run_id=run.id)
        print(run.status, end=" ")
        if run.status == "completed":
            print("Completed")
            break
        else:
            time.sleep(1)

    messages = Client.beta.threads.messages.list(thread_id=Thread.id)
    return messages

if OpTask=="answer":
    messages=runquery()
# print(messages.data[0].content[0].text.value)

## Print the OpenAI Answer ##

In [18]:
if OpTask=="check":
    s="** TOPIC ** : "+Query+"\n\n"
    for i in range(0,3):
        s=s+"**PAGE** : %s\n\n"%(metadata["texts"][topN[i]][1])
        s=s+"**URL** : %s\n\n"%(metadata["texts"][topN[i]][0])
elif OpTask=="answer":
    s="\n\n**Query** : "+Query+"\n\n"
    s=s+"**ANSWER** : "+messages.data[0].content[0].text.value
md(s)
md("\n\n")
Log.write(s)
Log.close()


** TOPIC ** : DOE Hydrogen Program

**PAGE** : Despicable Acts - Part 2

**URL** : https://halimgur.substack.com/p/despicable-acts-part-2

**PAGE** : The importance of elite consensus

**URL** : https://halimgur.substack.com/p/the-importance-of-elite-consensus

**PAGE** : ROGUE Age Accessory #1 - Population

**URL** : https://halimgur.substack.com/p/rogue-age-accessory-1-population

